<b> Fauzan Firdaus (23520011)<br> 
Nadya Aditama (23520039) </b>

# Flower Recognition Experiment
Pada notebook ini, akan dilakukan eksperimen mengenai rekognisi citra bunga dengan membandingkan deskriptor fitur ekstraksi gabungan dengan arsitektur model CNN.




## Deskripsi Singkat Dataset
![CaptureFlower](https://user-images.githubusercontent.com/15353477/98227599-14368f80-1f8a-11eb-9af1-549adb75746f.PNG)
Dataset yang digunakan adalah dataset **Oxford 17 Flower**, terdiri dari 1**7 spesies**, dimana setiap spesies terdiri dari **80 citra**, sehingga total datasetnya adalah **1.360 citra**. Karakteristik data tersebut adalah sebagai berikut


*   Citra memiliki ukuran yang bervariasi, namun perbedaan ukurannya tidak jauh (kisaran 500-800 piksel).
*   Skala citra bervariasi (portrait, landscape, dan square).
*   Posture dan lightning dari citra bervariasi.
*   Dalam satu citra bisa terdiri dari lebih dari satu bunga.



## Import Package
Berikut ini adalah package yang dibutuhkan untuk eksperimen ini

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import scipy
from IPython import display
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from skimage.feature import hog
from skimage.feature import local_binary_pattern
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from skimage.feature import greycomatrix, greycoprops
from sklearn.model_selection import KFold
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
import xgboost as xgb
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
%matplotlib inline

# 1. Descriptor Ekstraksi Fitur


## Load Image Data
Sebelum dilakukan load data, file citra dikategorikan dalam folder berdasarkan kelas. Kemudian di-load dengan code berikut

In [ ]:
DATADIR="gdrive/My Drive/Tugas Pattern Recognition/Tugas 1/17flowers"
CATEGORIES=["Bluebell","Buttercup","ColtsFoot",
            "Cowslip","Crocus","Daffodils",
            "Daisy","Dandelion","Fritillary",
            "Iris","LilyValley","Pansy",
            "Snowdrop","Sunflower","Tigerlily",
            "Tulip","Wildflower"]
training_data=[]
i = 1
for category in CATEGORIES:
    path = os.path.join(DATADIR,category)
    class_num=CATEGORIES.index(category)
    for img in os.listdir(path):
        try:
            img_array=cv2.imread(os.path.join(path,img))
            new_image=np.asarray(cv2.resize(img_array,(300,300)))
            training_data.append([new_image, class_num])
            print("Gambar termuat : ",i)
            display.clear_output(wait=True)
            i=i+1
        except Exception as e:
            pass## Load Image Data

Gambar termuat :  1360


## Split Train and Test Data
Dataset dibagi sebesar 80% data latih dan 20% data uji

In [ ]:
X = []
y = []
for image,label in training_data:
    X.append(image)
    y.append(label)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42,stratify=y)

Jumlah data latih dan data uji adalah sebagai berikut

In [ ]:
print("Jumlah data latih : ",np.shape(X_train)[0])
print("Jumlah data uji   : ",np.shape(X_test)[0]) 

Jumlah data latih :  1088
Jumlah data uji   :  272


## Preprocessing
Fitur yang akan dilihat dalam pengenalan citra bunga adalah fitur warna, bentuk, dan tekstur. Preprocessing yang dilakukan dalam citra tersebut adalah sebagai berikut
1.   Konversi ruang warna RGB ke Grayscale
2.   Konversi ruang warna RGB ke HSV



In [ ]:
def preprocessing1(arr):
    arr_prep=[]
    for i in range(np.shape(arr)[0]):
        img=cv2.cvtColor(arr[i], cv2.COLOR_BGR2GRAY)
        arr_prep.append(img)
    return arr_prep

def preprocessing2(arr):
    arr_prep=[]
    for i in range(np.shape(arr)[0]):
        image=cv2.cvtColor(arr[i], cv2.COLOR_BGR2GRAY)
        arr_prep.append(image)
    return arr_prep

def preprocessing3(arr):
    arr_prep=[]
    for i in range(np.shape(arr)[0]):
        image=cv2.cvtColor(arr[i], cv2.COLOR_BGR2HSV)
        arr_prep.append(image)
    return arr_prep

## Feature Extraction
Metode ekstraksi fitur yang digunakan dalam eksperimen ini adalah sebagai berikut
1.   HOG (Histogram of Oriented Gradient) sebagai deskriptor bentuk.
2.   GLCM (Gray Level Co-occurrence Matrix) sebagai deskriptor tesktur
3.   YCBcr (Color Histogram dari YCBcr) sebagai deskriptor warna



In [ ]:
def FtrExtractHOG(img):
    ftr,_=hog(img, orientations=9, pixels_per_cell=(50, 50),
            cells_per_block=(2, 2), visualize=True, multichannel=False, block_norm= 'L2')
    return ftr

def FtrExtractGLCM(img, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):
    glcm = greycomatrix(img, 
                        distances=dists, 
                        angles=agls, 
                        levels=lvl,
                        symmetric=sym, 
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in greycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
        
    return feature

def FtrExtractHSV(img):
    chans = cv2.split(img)
    features = []

    for chan in chans:
        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
        features.extend(hist)
    return np.array(features).flatten()

In [ ]:
def featureExtraction1(arr):
    arr_feature=[]
    for i in range(np.shape(arr)[0]):
        arr_feature.append(FtrExtractHOG(arr[i]))
    return arr_feature

properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
def featureExtraction2(arr):
    arr_feature=[]
    for i in range(np.shape(arr)[0]):
        arr_feature.append(FtrExtractGLCM(arr[i],props=properties))
    return arr_feature

def featureExtraction3(arr):
    arr_feature=[]
    for i in range(np.shape(arr)[0]):
        arr_feature.append(FtrExtractHSV(arr[i]))
    return arr_feature

## K-Fold Cross Validation

Validasi yang dilakukan menggunakan metode K-fold cross validation dengan K sebesar 5

In [ ]:
def kfold_1(X_train,y_train,K,stat):

  cvscores = []
  i = 1
  skf = KFold(n_splits=K)

  X_train_ = np.array(X_train)
  y_train = np.array(y_train)
  if (stat==1):
      RF = RandomForestClassifier(n_estimators=400, random_state = 42)
      for train_index,val_index in skf.split(X_train_):
        x_train_new,x_val=X_train_[train_index],X_train_[val_index]
        y_train_new,y_val=y_train[train_index],y_train[val_index]

        RF.fit(x_train_new, y_train_new)

        y_pred = RF.predict(x_val)
        acc = accuracy_score(y_val, y_pred)
        print("Fold ke",i," -> ",end=" ")
        print("%.3f%%"%(acc*100))
        i=i+1
        cvscores.append(acc)
  elif (stat==2):
      DTL = DecisionTreeClassifier(criterion="entropy", min_samples_split=20, random_state=42)
      for train_index,val_index in skf.split(X_train_):
        x_train_new,x_val=X_train_[train_index],X_train_[val_index]
        y_train_new,y_val=y_train[train_index],y_train[val_index]

        DTL.fit(x_train_new, y_train_new)

        y_pred = DTL.predict(x_val)
        acc = accuracy_score(y_val, y_pred)
        print("Fold ke",i," -> ",end=" ")
        print("%.3f%%"%(acc*100))
        i=i+1
        cvscores.append(acc)
  elif (stat==3):
      XG = xgb.XGBClassifier(subsample=0.8, learning_rate=0.1)
      for train_index,val_index in skf.split(X_train_):
        x_train_new,x_val=X_train_[train_index],X_train_[val_index]
        y_train_new,y_val=y_train[train_index],y_train[val_index]

        XG.fit(x_train_new, y_train_new)

        y_pred = XG.predict(x_val)
        acc = accuracy_score(y_val, y_pred)
        print("Fold ke",i," -> ",end=" ")
        print("%.3f%%"%(acc*100))
        i=i+1
        cvscores.append(acc)
  
  return cvscores

Preprocessing terhadap masing-masing jenis ekstraksi fitur

In [ ]:
X_trainp_shape=preprocessing1(X_train)
X_testp_shape=preprocessing1(X_test)

X_trainp_texture=preprocessing2(X_train)
X_testp_texture=preprocessing2(X_test)

X_trainp_color=preprocessing3(X_train)
X_testp_color=preprocessing3(X_test)

Ekstraksi fitur bentuk, tekstur, dan warna

In [ ]:
X_trainftr_shape=featureExtraction1(X_trainp_shape)
X_testftr_shape=featureExtraction1(X_testp_shape)

X_trainftr_texture=featureExtraction2(X_trainp_texture)
X_testftr_texture=featureExtraction2(X_testp_texture)

X_trainftr_color=featureExtraction3(X_trainp_color)
X_testftr_color=featureExtraction3(X_testp_color)

In [ ]:
X_trainftr_combined = np.concatenate((X_trainftr_shape,X_trainftr_texture,X_trainftr_color),axis=1)
X_testftr_combined = np.concatenate((X_testftr_shape,X_testftr_texture,X_testftr_color),axis=1)

Ukuran dimensi fitur citra adalah sebagai berikut 

In [ ]:
print("Dimensi fitur bentuk  : ",np.shape(X_trainftr_shape)[1])
print("Dimensi fitur tekstur : ",np.shape(X_trainftr_texture)[1])
print("Dimensi fitur warna   : ",np.shape(X_trainftr_color)[1])
print("Dimensi fitur gabungan: ",np.shape(X_trainftr_combined)[1])

Dimensi fitur bentuk  :  900
Dimensi fitur tekstur :  24
Dimensi fitur warna   :  768
Dimensi fitur gabungan:  1692


## Random Forest (RF)

In [ ]:
RFValidation = kfold_1(X_trainftr_combined,y_train,5,1)
print("Akurasi Validasi RF : %.3f%%" % (np.average(RFValidation)*100))

Fold ke 1  ->  58.716%
Fold ke 2  ->  64.220%
Fold ke 3  ->  61.468%
Fold ke 4  ->  67.742%
Fold ke 5  ->  55.760%
Akurasi Validasi RF : 61.581%


In [ ]:
RF = RandomForestClassifier(n_estimators=400, random_state = 42)
RF.fit(X_trainftr_combined, y_train)
y_pred_RF = RF.predict(X_testftr_combined)
print("Akurasi Testing RF : %.3f%%"% (accuracy_score(y_test, y_pred_RF)*100))

Akurasi Testing RF : 66.176%


Berikut adalah metrics evaluation dari Random Forest



In [ ]:
print(classification_report(y_test, y_pred_RF))
print(confusion_matrix(y_test, y_pred_RF))

              precision    recall  f1-score   support

           0       0.63      0.75      0.69        16
           1       0.69      0.69      0.69        16
           2       0.88      0.44      0.58        16
           3       0.44      0.50      0.47        16
           4       0.71      0.62      0.67        16
           5       0.69      0.56      0.62        16
           6       0.71      0.94      0.81        16
           7       0.85      0.69      0.76        16
           8       0.79      0.94      0.86        16
           9       0.75      0.56      0.64        16
          10       0.56      0.62      0.59        16
          11       0.72      0.81      0.76        16
          12       0.58      0.44      0.50        16
          13       0.62      0.81      0.70        16
          14       0.62      0.81      0.70        16
          15       0.50      0.31      0.38        16
          16       0.63      0.75      0.69        16

    accuracy              

## Decision Tree Learning (DTL)

In [ ]:
DTLValidation = kfold_1(X_trainftr_combined,y_train,5,2)
print("Akurasi Validasi DTL : %.3f%%" % (np.average(DTLValidation)*100))

Fold ke 1  ->  29.817%
Fold ke 2  ->  40.826%
Fold ke 3  ->  30.275%
Fold ke 4  ->  39.631%
Fold ke 5  ->  30.876%
Akurasi Validasi DTL : 34.285%


In [ ]:
DTL = DecisionTreeClassifier(criterion="entropy", min_samples_split=20, random_state=42)
DTL.fit(X_trainftr_combined, y_train)
y_pred_DTL = DTL.predict(X_testftr_combined)
print("Akurasi Testing DTL : %.3f%%"% (accuracy_score(y_test, y_pred_DTL)*100))

Akurasi Testing DTL : 35.294%


Berikut adalah metrics evaluation dari Decision Tree Learning

In [ ]:
print(classification_report(y_test, y_pred_DTL))
print(confusion_matrix(y_test, y_pred_DTL))

              precision    recall  f1-score   support

           0       0.47      0.44      0.45        16
           1       0.50      0.56      0.53        16
           2       0.20      0.19      0.19        16
           3       0.15      0.12      0.14        16
           4       0.43      0.38      0.40        16
           5       0.36      0.25      0.30        16
           6       0.75      0.56      0.64        16
           7       0.42      0.31      0.36        16
           8       0.73      0.50      0.59        16
           9       0.26      0.38      0.31        16
          10       0.50      0.62      0.56        16
          11       0.32      0.38      0.34        16
          12       0.12      0.12      0.12        16
          13       0.33      0.38      0.35        16
          14       0.21      0.38      0.27        16
          15       0.07      0.06      0.07        16
          16       0.50      0.38      0.43        16

    accuracy              

## XGBoost (XGB)

In [ ]:
XGValidation = kfold_1(X_trainftr_combined,y_train,5,3)
print("Akurasi Validasi XGB : %.3f%%" % (np.average(XGValidation)*100))

Fold ke 1  ->  61.468%
Fold ke 2  ->  68.807%
Fold ke 3  ->  65.596%
Fold ke 4  ->  70.507%
Fold ke 5  ->  53.917%
Akurasi Validasi XGB : 64.059%


In [ ]:
XG = xgb.XGBClassifier(subsample=0.8, learning_rate=0.1)
XG.fit(X_trainftr_combined, y_train)
y_pred_XG = XG.predict(X_testftr_combined)
print("Akurasi Testing XG : %.3f%%"%(accuracy_score(y_test, y_pred_XG)*100))

Akurasi Testing XG : 69.485%


Berikut adalah metrics evaluation dari XGBoost

In [ ]:
print(classification_report(y_test, y_pred_XG))
print(confusion_matrix(y_test, y_pred_XG))

              precision    recall  f1-score   support

           0       0.60      0.94      0.73        16
           1       0.75      0.75      0.75        16
           2       0.69      0.69      0.69        16
           3       0.47      0.56      0.51        16
           4       0.62      0.50      0.55        16
           5       0.82      0.56      0.67        16
           6       0.83      0.94      0.88        16
           7       0.92      0.75      0.83        16
           8       0.87      0.81      0.84        16
           9       1.00      0.75      0.86        16
          10       0.53      0.62      0.57        16
          11       0.87      0.81      0.84        16
          12       0.77      0.62      0.69        16
          13       0.73      0.69      0.71        16
          14       0.53      0.62      0.57        16
          15       0.40      0.38      0.39        16
          16       0.72      0.81      0.76        16

    accuracy              

# 2. Arsitektur CNN


Normalisasi Data

In [ ]:
X_train = np.array(X_train).astype('float32')
X_test = np.array(X_test).astype('float32')

mean_image = np.mean(X_train, axis = 0)

X_train -= mean_image
X_test -= mean_image

In [ ]:
y_train_binary = to_categorical(y_train)
y_test_binary = to_categorical(y_test)

## 2.a Inception-V3


Buat model inception tanpa pretrained model

In [ ]:
def create_inception_model():
  model = InceptionV3(weights=None, include_top=False, input_shape=(300,300,3), pooling='max')
  x = model.output
  predictions = Dense(17, activation='softmax')(x)
  myModel_scratch = Model(inputs=model.input, outputs=predictions, name='InceptionV3')

  opt = keras.optimizers.Adam(learning_rate=0.0001)
  myModel_scratch.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return myModel_scratch

Validasi dengan 5-fold cross validation

In [ ]:
def kfold_2(X_train,y_train,K):

  model_validation=create_inception_model()
  cvscores = []
  i = 1
  skf = KFold(n_splits=K)

  X_train_ = np.array(X_train)
  y_train = np.array(y_train)
  for train_index,val_index in skf.split(X_train_):
    x_train_new,x_val=X_train_[train_index],X_train_[val_index]
    y_train_new,y_val=y_train[train_index],y_train[val_index]

    model_validation.fit(x_train_new, y_train_new, validation_data = (x_val,y_val), epochs=20, verbose=0)

    scores = model_validation.evaluate(x_val, y_val, verbose=0)
    # print("Fold ke",i," -> ",scores[1]*100,"%")
    print("Fold ke",i," -> ",end=" ")
    print("%.3f%%"%(scores[1]*100))
    i=i+1
    cvscores.append(scores[1])

  return cvscores

In [ ]:
inception_validation = kfold_2(X_train,y_train_binary,5)
print("Akurasi Validasi CNN-Inception : %.3f%%" % (np.average(inception_validation)*100))

Fold ke 1  ->  53.670%
Fold ke 2  ->  65.596%
Fold ke 3  ->  86.239%
Fold ke 4  ->  84.332%
Fold ke 5  ->  91.244%
Akurasi Validasi CNN-Inception : 76.216%


Lakukan training data.

In [ ]:
model = create_inception_model()
model.fit(X_train, y_train_binary,epochs=20)

Epoch 1/20
34/34 [==============================] - 15s 444ms/step - loss: 2.9902 - accuracy: 0.1489
Epoch 2/20
34/34 [==============================] - 16s 457ms/step - loss: 2.1834 - accuracy: 0.3511
Epoch 3/20
34/34 [==============================] - 16s 462ms/step - loss: 1.8543 - accuracy: 0.4072
Epoch 4/20
34/34 [==============================] - 15s 449ms/step - loss: 1.6758 - accuracy: 0.4669
Epoch 5/20
34/34 [==============================] - 15s 444ms/step - loss: 1.4516 - accuracy: 0.5460
Epoch 6/20
34/34 [==============================] - 15s 446ms/step - loss: 1.3648 - accuracy: 0.5653
Epoch 7/20
34/34 [==============================] - 15s 454ms/step - loss: 1.2827 - accuracy: 0.5892
Epoch 8/20
34/34 [==============================] - 15s 451ms/step - loss: 1.1273 - accuracy: 0.6406
Epoch 9/20
34/34 [==============================] - 15s 448ms/step - loss: 0.9941 - accuracy: 0.6700
Epoch 10/20
34/34 [==============================] - 15s 449ms/step - loss: 0.9541 - accura

Berikut adalah metrics evaluation dari Inception-V3

In [ ]:
predictions = model.predict(X_test , batch_size = 32).argmax(axis = 1)
print(classification_report(y_test_binary.argmax(axis = 1), predictions))
print(confusion_matrix(y_test_binary.argmax(axis = 1), predictions))

              precision    recall  f1-score   support

           0       0.67      0.88      0.76        16
           1       0.74      0.88      0.80        16
           2       0.62      0.31      0.42        16
           3       0.33      0.25      0.29        16
           4       0.55      0.38      0.44        16
           5       0.70      0.44      0.54        16
           6       0.79      0.69      0.73        16
           7       0.61      0.69      0.65        16
           8       0.75      0.94      0.83        16
           9       0.68      0.81      0.74        16
          10       0.71      0.62      0.67        16
          11       0.58      0.88      0.70        16
          12       0.70      0.44      0.54        16
          13       0.56      0.56      0.56        16
          14       1.00      0.56      0.72        16
          15       0.38      0.56      0.45        16
          16       0.65      0.94      0.77        16

    accuracy              

In [ ]:
accuracy = model.evaluate(X_test, y_test_binary, verbose=1)
print('Test loss    :', accuracy[0])
print('Test accuracy: %.2f%%' % (accuracy[1]*100))

9/9 [==============================] - 1s 109ms/step - loss: 1.2583 - accuracy: 0.6360
Test loss    : 1.2583361864089966
Test accuracy: 63.60%


## 2.b Custom Architecture (Model sendiri)

In [ ]:
def create_model(lr = 0.0001):
  model = Sequential([
      Conv2D(32,(3,3),input_shape=(300,300,3), padding='same', activation="relu"),
      Conv2D(32,(3,3), padding='same', activation="relu"),
      MaxPooling2D(3,3),
      Conv2D(64,(3,3), padding='same', activation="relu"),
      Conv2D(64,(3,3), padding='same', activation="relu"),
      MaxPooling2D(3,3),
      Conv2D(128,(3,3), padding='same', activation="relu"),
      Conv2D(256,(3,3), padding='same', activation="relu"),
      MaxPooling2D(3,3),
      Conv2D(256,(3,3), padding='same', activation="relu"),
      Flatten(),
      Dense(1024, activation="relu"),
      Dropout(0.5),
      Dense(512, activation="relu"),
      Dense(17, activation="softmax")
  ])
  opt = keras.optimizers.Adam(learning_rate=lr)
  model.compile(optimizer=opt,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model

Lakukan 5-fold cross validation dengan berbeda-beda learning rate.

In [ ]:
from sklearn.model_selection import KFold

cvscores = []
model_param = [0,0,0,0,0,0]
n_split=5

i = 1

X_train_ = np.array(X_train)
for train_index,val_index in KFold(n_split).split(X_train_):
  x_train_new,x_val=X_train_[train_index],X_train_[val_index]
  y_train_new,y_val=y_train_binary[train_index],y_train_binary[val_index]

  lr_score=[]
  print("Fold ke ",i)
  learning_rate = [0.1,0.01,0.001,0.0001,0.00001,0.000001]
  j=0
  for lr in learning_rate:
    #simpen model dengan learning rate masing2
    model_param[j]=create_model(lr=lr)
    model_param[j].fit(x_train_new, y_train_new, validation_data = (x_val,y_val), epochs=20, verbose=0)

    #ngambil score
    scores = model_param[j].evaluate(x_val, y_val, verbose=0)
    print('learning rate ',lr,"= %.2f%%" % (scores[1]*100))
    lr_score.append(scores[1]*100)
    j=j+1

  i=i+1
  cvscores.append(lr_score)

Fold ke  1
learning rate  0.1 = 5.05%
learning rate  0.01 = 1.83%
learning rate  0.001 = 47.71%
learning rate  0.0001 = 63.76%
learning rate  1e-05 = 51.38%
learning rate  1e-06 = 27.06%
Fold ke  2
learning rate  0.1 = 5.05%
learning rate  0.01 = 4.59%
learning rate  0.001 = 48.62%
learning rate  0.0001 = 63.30%
learning rate  1e-05 = 60.55%
learning rate  1e-06 = 26.15%
Fold ke  3
learning rate  0.1 = 5.96%
learning rate  0.01 = 3.67%
learning rate  0.001 = 47.25%
learning rate  0.0001 = 62.84%
learning rate  1e-05 = 56.42%
learning rate  1e-06 = 20.64%
Fold ke  4
learning rate  0.1 = 5.53%
learning rate  0.01 = 3.69%
learning rate  0.001 = 40.55%
learning rate  0.0001 = 62.67%
learning rate  1e-05 = 56.22%
learning rate  1e-06 = 29.95%
Fold ke  5
learning rate  0.1 = 5.07%
learning rate  0.01 = 3.23%
learning rate  0.001 = 48.39%
learning rate  0.0001 = 58.06%
learning rate  1e-05 = 48.85%
learning rate  1e-06 = 21.20%


In [ ]:
print("----------------- Rata-rata Seluruhnya ----------------------")
print("learning rate 0.1     = %.2f%% (+/- %.2f%%)" % (np.mean(cvscores, axis=0)[0], np.std(cvscores, axis=0)[0])) 
print("learning rate 0.01    = %.2f%% (+/- %.2f%%)" % (np.mean(cvscores, axis=0)[1], np.std(cvscores, axis=0)[1]))
print("learning rate 0.001   = %.2f%% (+/- %.2f%%)" % (np.mean(cvscores, axis=0)[2], np.std(cvscores, axis=0)[2]))
print("learning rate 0.0001  = %.2f%% (+/- %.2f%%)" % (np.mean(cvscores, axis=0)[3], np.std(cvscores, axis=0)[3]))
print("learning rate 0.00001   = %.2f%% (+/- %.2f%%)" % (np.mean(cvscores, axis=0)[4], np.std(cvscores, axis=0)[4]))
print("learning rate 0.000001  = %.2f%% (+/- %.2f%%)" % (np.mean(cvscores, axis=0)[5], np.std(cvscores, axis=0)[5]))

----------------- Rata-rata Seluruhnya ----------------------
learning rate 0.1     = 5.33% (+/- 0.37%)
learning rate 0.01    = 3.40% (+/- 0.90%)
learning rate 0.001   = 46.50% (+/- 3.02%)
learning rate 0.0001  = 62.13% (+/- 2.07%)
learning rate 0.00001   = 54.68% (+/- 4.12%)
learning rate 0.000001  = 25.00% (+/- 3.57%)


In [ ]:
modelfixed = model_param[3]

predictions = modelfixed.predict(X_test , batch_size = 32).argmax(axis = 1)

In [ ]:
print('Model evaluation ',modelfixed.evaluate(X_test,y_test_binary))

9/9 [==============================] - 0s 42ms/step - loss: 1.5640 - accuracy: 0.6434
Model evaluation  [1.5639725923538208, 0.6433823704719543]


In [ ]:
model_predict = modelfixed.predict(X_test)
class_id = np.argmax(model_predict,axis=1)

Berikut adalah metrics evaluation dari arsitektur CNN model yang kami bangun.

In [ ]:
print(classification_report(y_test_binary.argmax(axis = 1), predictions))
print(confusion_matrix(y_test_binary.argmax(axis = 1), predictions))

              precision    recall  f1-score   support

           0       0.44      1.00      0.62        16
           1       0.64      0.88      0.74        16
           2       0.53      0.56      0.55        16
           3       0.62      0.31      0.42        16
           4       0.62      0.31      0.42        16
           5       0.86      0.38      0.52        16
           6       0.71      0.75      0.73        16
           7       0.73      0.69      0.71        16
           8       1.00      0.50      0.67        16
           9       1.00      0.81      0.90        16
          10       0.59      0.81      0.68        16
          11       1.00      0.69      0.81        16
          12       0.42      0.50      0.46        16
          13       0.67      0.62      0.65        16
          14       0.70      1.00      0.82        16
          15       0.38      0.31      0.34        16
          16       0.72      0.81      0.76        16

    accuracy              